In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def baseline_model(input_shape):
    model = Sequential([
        Conv2D(16, (4, 4), activation='relu', input_shape=input_shape),
        Conv2D(16, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def residual_model(input_shape):
    model = Sequential([
        Conv2D(16, (2, 2), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(16, (3, 3), activation='relu'),
        Conv2D(16, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(16, (3, 3), activation='relu'),
        Conv2D(16, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(92, (3, 3), activation='relu'),
        Conv2D(92, (3, 3), activation='relu'),
        Dropout(0.25),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        Dropout(0.25),
        BatchNormalization(),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model


In [ ]:
train_data_dir = '/path/to/train/dataset'
validation_data_dir = '/path/to/validation/dataset'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(101, 101),
    batch_size=32,
    class_mode='binary')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(101, 101),
    batch_size=32,
    class_mode='binary')


In [ ]:
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
baseline_history = baseline_model.fit(train_generator, validation_data=validation_generator, epochs=10)


In [ ]:
residual_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
residual_history = residual_model.fit(train_generator, validation_data=validation_generator, epochs=10)

In [ ]:
# Calculate ROC and AUC for baseline model
baseline_y_pred = baseline_model.predict(validation_generator)
baseline_fpr, baseline_tpr, _ = roc_curve(validation_generator.classes, baseline_y_pred)
baseline_auc = roc_auc_score(validation_generator.classes, baseline_y_pred)


In [ ]:
# Calculate ROC and AUC for residual model
residual_y_pred = residual_model.predict(validation_generator)
residual_fpr, residual_tpr, _ = roc_curve(validation_generator.classes, residual_y_pred)
residual_auc = roc_auc_score(validation_generator.classes, residual_y_pred)

In [ ]:
# Get predictions and ground truth labels for classification report
baseline_y_pred_classes = np.argmax(baseline_y_pred, axis=1)

In [ ]:
residual_y_pred_classes = np.argmax(residual_y_pred, axis=1)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(baseline_fpr, baseline_tpr, label=f'Baseline Model (AUC = {baseline_auc:.2f})')
plt.plot(residual_fpr, residual_tpr, label=f'Residual Model (AUC = {residual_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


In [ ]:
print("Baseline Model Classification Report:")
print(classification_report(validation_generator.classes, baseline_y_pred_classes))


In [ ]:
print("Residual Model Classification Report:")
print(classification_report(validation_generator.classes, residual_y_pred_classes))